In [1]:
### spark command
!sudo /usr/local/spark/sbin/start-master.sh

spark: /usr/local/spark


### Standalone Mode Cluster version 2.0.2
* * *

``` sh
docker network create spark_network
```

##### Create Master
``` sh
docker run -it --net spark_network --name master -p 8080:8080 p7hb/docker-spark:2.0.2

/usr/local/spark/sbin/start-master.sh
```

##### Create Worker
``` sh
docker run -it --net spark_network --name worker1 p7hb/docker-spark:2.0.2

/usr/local/spark/sbin/start-slave.sh master:7077
```

##### Config Spark Env 

add the following into ./conf/spark-env.sh

- export SPARK_MASTER_IP=master
- export SPARK_MASTER_PORT=7077
- export SPARK_WORKER_CORES=1
- export SPARK_WORDER_INSTANCES=1
- export SPARK_WORKER_MEMORY=3g

##### Add Slaves 

Update conf/slaves

- localhost
- worker1

##### Run Example

``` sh
./bin/spark-submit --class org.apache.spark.examples.SparkPi --master spark://master:7077 $SPARK_HOME/examples/src/main/python/pi.py 10
```

##### Another way

``` sh
sbin/start-all.sh
```

and use 
``` sh
netstat -nat
```

monitor port 7077 process

##### Import Required Lib

add /usr/local/spark/python/lib/py4j-0.10.3-src.zip to sys.path

In [ ]:
import os
# make sure pyspark tells workers to use python3 not 2 if both are installed
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

import pyspark
conf = pyspark.SparkConf()

conf.setMaster("spark://master:7077")
# conf.set("spark.executor.memory", "4g")
# conf.set("spark.core.connection.ack.wait.timeout", "1200")
conf.setAppName("My App")

# create the context
sc = pyspark.SparkContext(conf=conf)

##### mesos master

``` sh
docker run -d --net spark_network --name master --user root -e GRANT_SUDO=yes -p 5050:5050 jupyter/all-spark-notebook:latest

mesos-master --work_dir=/var/lib/mesos/
```

##### mesos slave

``` sh
docker run -d --name=slave2 --net spark_network -e GRANT_SUDO=yes --user root jupyter/all-spark-notebook

mesos-slave --master=master:5050 --work_dir=/var/lib/mesos/
```

In [14]:
!echo $PYSPARK_PYTHON